In [152]:
#imports and constants
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.neighbors import NearestNeighbors

KNN = 5

In [153]:
def fvecs_read(filename, c_contiguous=True):
    fv = np.fromfile(filename, dtype=np.float32)
    if fv.size == 0:
        return np.zeros((0, 0))
    dim = fv.view(np.int32)[0]
    assert dim > 0
    fv = fv.reshape(-1, 1 + dim)
    if not all(fv.view(np.int32)[:, 0] == dim):
        raise IOError("Non-uniform vector sizes in " + filename)
    fv = fv[:, 1:]
    if c_contiguous:
        fv = fv.copy()
    return fv

def fvecs_write(data, filename):
    n, d = data.shape
    data_to_file = np.empty((n, d+1), np.float32)
    data_to_file.view(np.int32)[:, 0] = d
    data_to_file[:, 1:] = data
    data_to_file.tofile(filename)

In [154]:
data = fvecs_read("audio_base.fvecs") #read in the data
data

array([[54343., 29776., 61789., ..., 48916., 51402., 58398.],
       [61316., 65420., 56087., ..., 64560., 52845., 54574.],
       [40361., 62775., 60291., ..., 46604., 59174., 54336.],
       ...,
       [43378., 31941., 63146., ..., 49708., 56622., 58992.],
       [55945., 66552., 53190., ..., 55152., 58037., 55757.],
       [65975., 64302., 53143., ..., 58365., 57474., 53535.]],
      dtype=float32)

In [155]:
trainData, queryData = train_test_split(data, test_size=0.1) # split the data into training and test

In [191]:
%%time
df = pd.Series((q for q in queryData[0:2])).to_frame(name="queryPoint")
df['KNN'] = ""
# df['rest'] = ""
# setOfIndicies = set(range(0,trainData.shape[0]))
neigh = NearestNeighbors(n_neighbors=KNN)
neigh.fit(trainData) 
NearestNeighbors(algorithm='auto', leaf_size=30, metric='euclidean')
#for each query item, compute the KNN distances
for index, row in df.iterrows():
    row['KNN'] = neigh.kneighbors([row['queryPoint']])[1][0] #store the indicies of the KNN's in KNN field
#     row['rest'] = setOfIndicies - set(row['KNN']) #compute the rest by set difference

CPU times: user 702 ms, sys: 13.1 ms, total: 715 ms
Wall time: 716 ms


In [192]:

df

,queryPoint,KNN
0,"[46319.0, 45374.0, 62284.0, 59239.0, 55504.0, ...","[38209, 5709, 600, 33236, 8180]"
1,"[79884.0, 56648.0, 48408.0, 52091.0, 56305.0, ...","[31257, 20534, 11323, 17796, 36734]"
